# A Step-by-Step Guide to Holoclean example

Noisy and erroneous data is a major bottleneck in analytics. Data cleaning and repairing account for about 60% of the work of data scientists. To address this bottleneck, we recently introduced HoloClean, a semi-automated data repairing framework that relies on statistical learning and inference to repair errors in structured data. In HoloClean, we build upon the paradigm of weak supervision and demonstrate how to leverage diverse signals, including user-defined heuristic rules (such as generalized data integrity constraints) and external dictionaries, to repair erroneous data. 

In this post, we walk through the process of implementing Holoclean, by creating a simple end-to-end example.

# Setup

Firstly, we import all the module from Holoclean that we will use.

In [1]:
from holoclean.holoclean import HoloClean, Session
from holoclean.errordetection.errordetector import ErrorDetectors
from holoclean.featurization.featurizer import SignalInit, SignalCooccur, SignalDC
from holoclean.learning.accuracy import Accuracy
from time import time as t

##   Initialization
In this part, we create the Holoclean and Session object that we will use for this example.

In [2]:
        holo_obj = HoloClean()
        session = Session("Session", holo_obj) 
        print "Testing started :"+str(t())
        

Testing started :1517342571.36


/home/gmichalo/anaconda/envs/python27/lib/python2.7/site-packages/sqlalchemy/dialects/mysql/base.py:1576: Warning: '@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead
  cursor.execute('SELECT @@tx_isolation')


## Read Input and DC from file
Test data and the Denial Constraints will be read using the Session's ingestor.
After ingesting the test data will be loaded into MySQL tables along with entries in the a metadata table.

In [3]:
        fx = open('execution_time.txt', 'w')
        list_time = []
        start_time = t()
        
        session.ingest_dataset("test/inputDatabase.csv")
        #session.ingest_dataset("test/test.csv")
        d = t()-start_time
        list_time.append(d)
        holo_obj.logger.info('ingest csv time: '+str(d)+'\n')
        fx.write('ingest csv time: '+str(d)+'\n')
        print 'Init table'
        sql = holo_obj.dataengine.get_table_to_dataframe("Init", session.dataset)
        sql.select('index','ProviderNumber','HospitalName', 'Address1').show()
        print 'ingest csv time: '+str(d)+'\n'
        
        start_time = t()
        
        session.denial_constraints("test/inputConstraint.txt")
        #session.denial_constraints("test/dc1.txt")
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('read denial constraints time: '+str(d)+'\n')
        fx.write('read denial constraints time: '+str(d)+'\n')
        print 'read denial constraints time: '+str(d)+'\n'


Init table
+-----+--------------+--------------------+--------------------+
|index|ProviderNumber|        HospitalName|            Address1|
+-----+--------------+--------------------+--------------------+
|    1|         10018|CALLAHAN EYE FOUN...|1720 UNIVERSITY BLVD|
|    2|         10018|CALLAHAN EYE FOUN...|1720 UNIVERSITY BLVD|
|    3|         10018|CALLAHAN EYE FOUN...|1720 UNIVERSITY BLVD|
|    4|         10018|CALLAHAN EYE FOUN...|1720 UNIVERSITY BLVD|
|    5|         10018|CALLAHAN EYE FOUN...|1720 UNIVERSITY BLVD|
|    6|         10018|CALLAHAN EYE FOUN...|1720 UNIVERSITY BLVD|
|    7|         10018|CALLAHAN EYE FOUN...|1720 UNIVERSITY BLVD|
|    8|         10018|CALLAHAN EYE FOUN...|1720 UNIVERSITY BLVD|
|    9|         10019|HELEN KELLER MEMO...|1300 SOUTH MONTGO...|
|   10|         10019|HELEN KELLER MEMO...|1300 SOUTH MONTGO...|
|   11|         10019|HELEN KELLER MEMO...|1300 SOUTH MONTGO...|
|   12|         10019|HELEN KELLER MEMO...|1300 SOUTH MONTGO...|
|   13|       

## Error Detection
In this part, we create the error detection. The output of this part is the C_dk table that contains all the noisy cells and the C_Clean table that contains the clean cells

In [4]:
        start_time = t()
        err_detector = ErrorDetectors(session.Denial_constraints, holo_obj.dataengine,
                                      holo_obj.spark_session, session.dataset)
        session.add_error_detector(err_detector)
        session.ds_detect_errors()
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('error dectection time: '+str(d)+'\n')
        fx.write('error dectection time: '+str(d)+'\n')
        
        print 'Clean table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_clean", session.dataset)
        sql.show()
        print 'Don\'t know table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_dk", session.dataset)
        sql.show()
        print 'error dectection time: '+str(d)+'\n'

Clean table
+---+----------------+
|ind|            attr|
+---+----------------+
| 10|        Address1|
|140|           Score|
|156|      CountyName|
|104|    HospitalType|
|175|    HospitalName|
|106|        Address3|
|109|    HospitalType|
| 18|      CountyName|
|107|    HospitalName|
|163|  ProviderNumber|
|166|   HospitalOwner|
|110|    HospitalName|
|201|        Address1|
|183|EmergencyService|
|201|          Sample|
|110|    HospitalType|
|205|    HospitalType|
|219|  ProviderNumber|
|146|        Address1|
|253|           Score|
+---+----------------+
only showing top 20 rows

Don't know table
+---+-------+
|ind|   attr|
+---+-------+
|199|   City|
|658|   City|
|228|ZipCode|
|702|   City|
|533|ZipCode|
|805|   City|
|466|ZipCode|
|460|   City|
|314|ZipCode|
|311|ZipCode|
|596|   City|
|325|ZipCode|
|853|ZipCode|
|272|  State|
|882|   City|
|616|  State|
|643|ZipCode|
|893|  State|
|529|   City|
|206|   City|
+---+-------+
only showing top 20 rows

error dectection time: 266.1556

## Domain Pruning
In this part, we prune the domain. The output of this part is the possible_values tables that contains all the possible values for each cell

In [5]:
        start_time = t()
        pruning_threshold = 0.5
        session.ds_domain_pruning(pruning_threshold)
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('domain pruning time: '+str(d)+'\n')
        fx.write('domain pruning time: '+str(d)+'\n')
        
        print 'Possible Values table'
        sql = holo_obj.dataengine.get_table_to_dataframe("Possible_values", session.dataset)
        sql.show()
        print 'domain pruning time: '+str(d)+'\n'

Possible Values table
+---+-----------+--------------------+--------+---------+
|tid|  attr_name|            attr_val|observed|data_type|
+---+-----------+--------------------+--------+---------+
|456|       City|           SHEFFIELD|       1|   String|
|456|       City|          BIRMINGHAM|       0|   String|
|456|      State|                  AL|       1|   String|
|456|    ZipCode|               35660|       1|   String|
|456|PhoneNumber|          2563864556|       1|   String|
|456|  Condition|           Pneumonia|       1|   String|
|456|MeasureCode|               PN-5C|       1|   String|
|456|MeasureName|Pneumonia Patient...|       1|   String|
|456|   Stateavg|            AL_PN-5C|       1|   String|
|457|       City|           SHEFFIELD|       1|   String|
|457|      State|                  AL|       1|   String|
|457|    ZipCode|               35660|       1|   String|
|457|PhoneNumber|          2563864556|       1|   String|
|457|  Condition|           Pneumonia|       1|   

# Featurization

In this part, we implement the featurization module of holoclean. We choose the signals that we want to use and the output of this part is the featurization table that contains the factors that we will use

## Initial Value Signal 

In [6]:
        start_time = t()
        start_time1 = t()
        initial_value_signal = SignalInit(session.Denial_constraints, holo_obj.dataengine,
                                          session.dataset)
        session.add_featurizer(initial_value_signal)
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('init signal time: '+str(d)+'\n')
        fx.write('init signal time: '+str(d)+'\n')
        print 'init signal time: '+str(d)+'\n'

init signal time: 0.00117802619934



## Co-occurence Signal 

In [7]:
        start_time = t()
   
        statistics_signal = SignalCooccur(session.Denial_constraints, holo_obj.dataengine,
                                          session.dataset)
        session.add_featurizer(statistics_signal)
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('cooccur signal time: '+str(d)+'\n')
        fx.write('cooccur signal time: '+str(d)+'\n')
        print 'cooccur signal time: '+str(d)+'\n'

cooccur signal time: 0.000410079956055



## DC Signal

In [8]:
        start_time = t()
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('dc signal time: '+str(d)+'\n')
        fx.write('dc signal time: '+str(d)+'\n')
        print 'dc signal time: '+str(d)+'\n'
        start_time = t()
        dc_signal = SignalDC(session.Denial_constraints, holo_obj.dataengine, session.dataset)
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('dc featurize time: '+str(d)+'\n')
        fx.write('dc featurize time: '+str(d)+'\n')
        print 'dc featurize time: '+str(d)+'\n'
        session.add_featurizer(dc_signal)

dc signal time: 3.38554382324e-05

dc featurize time: 0.00020694732666



We use the signals that we choose in the previous steps. The output of this part is the featurization table that contains the factors that we will use in the next step.

In [9]:
        session.ds_featurize()
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('total featurization time: '+str(d)+'\n')
        fx.write('total featurization time: '+str(d)+'\n')
        print 'Feature table'
        sql = holo_obj.dataengine.get_table_to_dataframe("Feature", session.dataset)
        sql.show()
        
        print 'total featurization time: '+str(d)+'\n'

adding weight_id to feature table...
adding weight_id to feature table is finished
Feature table
+---------+--------+--------------+--------------------+--------------------+----+---------+
|var_index|rv_index|       rv_attr|        assigned_val|             feature|TYPE|weight_id|
+---------+--------+--------------+--------------------+--------------------+----+---------+
|        1|     216|     Condition|Surgical Infectio...|Init=Surgical Inf...|init|         |
|        2|     216|   MeasureCode|          SCIP-INF-4|     Init=SCIP-INF-4|init|         |
|        3|     216|   MeasureName|all heart surgery...|Init=all heart su...|init|         |
|        4|     216|      Stateavg|       AL_SCIP-INF-4|  Init=AL_SCIP-INF-4|init|         |
|        5|     217|ProviderNumber|               10011|          Init=10011|init|         |
|        6|     217|          City|          BIRMINGHAM|     Init=BIRMINGHAM|init|         |
|        7|       1|ProviderNumber|               10018|          

#  Learning
In the learning phase, we create a wrapper for numbskull that we will use for the gibbs sampling. The output of this part is the new weight table.

In [10]:
        start_time = t()
        session._numskull()
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('numbskull time: '+str(d)+'\n')
        fx.write('numbskull time: '+str(d)+'\n')
        print 'numbskull time: '+str(d)+'\n'
        start_time = t()

numbskull is starting
wrapper is starting
wrapper is finished
1
numbskull is finished
adding weight is finished is finished
numbskull time: 324.640882969



In this part, we use the new weight, to learn the probabilities for each value for the cells


In [11]:
       session.ds_repair()
        d = t() - start_time
        list_time.append(d)
        holo_obj.logger.info('repair time: '+str(d)+'\n')
        fx.write('repair time: '+str(d)+'\n')
        print 'repair time: '+str(d)+'\n'

        holo_obj.logger.info('Total time: ' + str(sum(list_time)) + '\n')
        fx.write('Total time: ' + str(sum(list_time)) + '\n')
        print 'Total time: ' + str(sum(list_time)) + '\n'

        fx.close()

IndentationError: unexpected indent (<ipython-input-11-25901fef5a54>, line 2)